### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
unique_players = purchase_data["SN"].unique()
print("Total Players: "+str(unique_players.size))

Total Players: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
import statistics;
import locale;

avg = statistics.mean(purchase_data["Price"])
total_revenue = purchase_data["Price"].sum()

locale.setlocale( locale.LC_ALL, '' )

purchase_analysis = pd.DataFrame({
    "Number of Unique Items":[purchase_data["Item ID"].unique().size],
    "Average Price":[locale.currency(avg)],
    "Number of Purchases":[purchase_data["Purchase ID"].size],
    "Total Revenue":[locale.currency(total_revenue, grouping=True)]
})

purchase_analysis.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#this will get the correct answer, but the code is rather a mess.  Check the next cell for a more elegant solution.
total = 0
numM = 0
numF = 0
numI = 0

for player in unique_players:
    found = False
    for i in range(len(purchase_data["SN"])): 
        if (not found):
            if (purchase_data["SN"][i] == player):
                if (purchase_data["Gender"][i] == "Male"):
                    numM += 1
                elif (purchase_data["Gender"][i] == "Female"):
                    numF += 1
                else:
                    numI += 1
                found = True

total = numM + numF + numI
print(total)

data = {'Total Count': [numM, numF,numI], 
        'Percentage of Players': ["{0:.2f}".format(numM/total*100), "{0:.2f}".format(numF/total*100),"{0:.2f}".format(numI/total*100)]}

gender_demographics = pd.DataFrame(data,index = ['Male','Female','Other'])
gender_demographics.head()


576


,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other,11,1.91


In [5]:
#more elegant solution
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.sort_values("SN", inplace = True)
purchase_data.drop_duplicates(subset ="SN", keep="last",inplace=True)

count = purchase_data["Gender"].value_counts()

numM = count["Male"]
numF = count["Female"]
numI= count["Other / Non-Disclosed"]
total = numM + numF + numI

data = {'Total Count': [numM, numF,numI], 
        'Percentage of Players': ["{0:.2f}".format(numM/total*100), "{0:.2f}".format(numF/total*100),"{0:.2f}".format(numI/total*100)]}

gender_demographics = pd.DataFrame(data,index = ['Male','Female','Other'])
gender_demographics.head()

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
def find_gender_params(gender, numG):
    gender_data = {}    
    
    gd = purchase_data.loc[purchase_data["Gender"] == gender, :]
    gender_data["Purchase Count"] = len(list(gd["Price"]))
    avg = locale.currency(statistics.mean(list(gd["Price"])))
    gender_data["Average Purchase Price"] = locale.currency(statistics.mean(list(gd["Price"])))
    gender_data["Total Purchase Value"] = locale.currency(sum(list(gd["Price"])))
    gender_data["Avg Total Purchase per Person"] = locale.currency(sum(list(gd["Price"]))/numG)
    
    return gender_data
 

import statistics
import locale
locale.setlocale( locale.LC_ALL, '' )
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

gender_stats = [find_gender_params("Female", numF), find_gender_params("Male", numM), find_gender_params("Other / Non-Disclosed", numI)]

gender_stats_df = pd.DataFrame(gender_stats, index = ['Male','Female','Other / Non-Disclosed'])
gender_stats_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Male,113,$3.20,$361.94,$4.47
Female,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [37]:
agesbins = [0,9,14,19,24,29,34,39,1000]
ageslabels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
 
demo_data = purchase_data.sort_values(by=['SN'])
demo_data = demo_data.drop_duplicates(subset=["SN"],keep='first')
demo_data["Age Demo"] = pd.cut(demo_data["Age"], agesbins, labels=ageslabels)
len(demo_data['SN'])

576

In [69]:
grouped_demo = demo_data.groupby(['Age Demo'])
gd = grouped_demo.count().rename(columns={"Purchase ID": "Total Count"})
list(gd.columns)
for c in list(gd.columns):
    if c != "Total Count":
        del gd[c]

gd['Percentage of Players'] = round(gd['Total Count'] / len(demo_data['SN']) * 100,2)

age = gd.copy()
gd

,Total Count,Percentage of Players
Age Demo,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [70]:
purchase_age = purchase_data.copy()
purchase_age["Age Demo"] = pd.cut(purchase_data["Age"], agesbins, labels=ageslabels)
grouped_purchase = purchase_age.groupby(['Age Demo'])

#create some series
gd = grouped_purchase.count()
purchase_count = gd['SN']

gd = grouped_purchase.mean()
avg_purchase_price = gd['Price']

gd = grouped_purchase.sum()
total_purchase_value = gd['Price']

#glue the series together in a dictionary
frame = { 'Purchase Count': purchase_count, 'Average Purchase Price': avg_purchase_price , 'Total Purchase Value':total_purchase_value} 

#make that dictionary a datafram
result = pd.DataFrame(frame) 
result['Avg Total Purchase per Person'] = result['Total Purchase Value']/age['Total Count']
result.head(10)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Demo,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

